In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-mar-2021/train.csv
/kaggle/input/tabular-playground-series-mar-2021/test.csv


In [2]:
train_path = "/kaggle/input/tabular-playground-series-mar-2021/train.csv"
test_path = "/kaggle/input/tabular-playground-series-mar-2021/test.csv"

In [3]:
import hashlib

In [4]:
s="A"
int(hashlib.sha256(s.encode('utf-8')).hexdigest(), 16) % 10**2

45

In [5]:
def hash_cat(s):
    return int(hashlib.sha256(s.encode('utf-8')).hexdigest(), 16) % 10**3

In [6]:
# x = ['A', "b"]


In [7]:
# x.index('c')

In [8]:
cat_dict = []
def hash_cat_two(s):
    if s not in cat_dict:
        cat_dict.append(s)
    return cat_dict.index(s)

In [9]:
import pandas as pd

In [10]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)


In [11]:
len(df_train)

300000

In [12]:
df_train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,0,A,I,A,B,B,BI,A,S,Q,...,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0
1,1,A,I,A,A,E,BI,K,W,AD,...,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0
2,2,A,K,A,A,E,BI,A,E,BM,...,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0
3,3,A,K,A,C,E,BI,A,Y,AD,...,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0
4,4,A,I,G,B,E,BI,C,G,Q,...,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1


In [13]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

In [14]:
df_train.columns

Index(['id', 'cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15',
       'cat16', 'cat17', 'cat18', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4',
       'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'target'],
      dtype='object')

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
# ?train_test_split

In [17]:
cat_col = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15',
       'cat16', 'cat17', 'cat18']
cont_col = ['cont0', 'cont1', 'cont2', 'cont3', 'cont4',
       'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10']

In [18]:
for col in cat_col:
    encoder = LabelEncoder()
    # encoder.fit(list(df_train[col]) + list(df_test[col]))
    encoder.fit(list(df_train[col].values) + list(df_test[col].values))
    df_train[col] = encoder.transform(df_train[col].values)
    df_test[col] = encoder.transform(df_test[col].values)
    print("done ", col)

done  cat0
done  cat1
done  cat2
done  cat3
done  cat4
done  cat5
done  cat6
done  cat7
done  cat8
done  cat9
done  cat10
done  cat11
done  cat12
done  cat13
done  cat14
done  cat15
done  cat16
done  cat17
done  cat18


In [19]:
X = df_train.drop(["id", "target"], axis=1)
y = df_train["target"]


In [20]:
X.shape

(300000, 30)

In [21]:
y.shape

(300000,)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=111)

In [23]:
import lightgbm as gbm

In [24]:
df_train["cat0"].unique()

array([0, 1])

In [25]:
train_data = gbm.Dataset(X_train, label=y_train, categorical_feature=cat_col)

In [26]:
validation_data = gbm.Dataset(X_test, label=y_test, reference=train_data)

In [27]:
lgbm_params = {
    'boosting': 'gbdt',          # dart (drop out trees) often performs better
    'application': 'binary',     # Binary classification
    'learning_rate': 0.02,       # Learning rate, controls size of a gradient descent step
    'min_data_in_leaf': 20,      # Data set is quite small so reduce this a bit
    'feature_fraction': 0.7,     # Proportion of features in each boost, controls overfitting
    'num_leaves': 100,            # Controls size of tree since LGBM uses leaf wise splits
    'metric': 'auc',  # Area under ROC curve as the evaulation metric
}

https://www.kaggle.com/sayantansadhu/lgbm-with-hyper-parameter-tuning-using-optuna

In [28]:
lgbm_params = {
    'boosting': 'gbdt',          # dart (drop out trees) often performs better
    'application': 'binary',     # Binary classification
    'learning_rate': 0.02,       # Learning rate, controls size of a gradient descent step
    'min_data_in_leaf': 100,      # Data set is quite small so reduce this a bit
    'feature_fraction': 0.7,     # Proportion of features in each boost, controls overfitting
    'metric': 'binary_logloss',  # Area under ROC curve as the evaulation metric,
    'lambda_l1': 5e-05, 
     'lambda_l2': 1.35e-08, 
     'num_leaves': 216, 
     'feature_fraction': 0.7458519562366559, 
     'bagging_fraction': 0.7835116194444349, 
     'bagging_freq': 2, 
     'min_child_samples': 95
}

# {'lambda_l1': 5.7529066102502985e-05, 
#  'lambda_l2': 1.3522520634637266e-08, 
#  'num_leaves': 216, 
#  'feature_fraction': 0.7458519562366559, 
#  'bagging_fraction': 0.7835116194444349, 
#  'bagging_freq': 2, 
#  'min_child_samples': 95}

In [29]:
num_round = 1000
bst = gbm.train(lgbm_params, train_data, num_round, valid_sets=[validation_data], early_stopping_rounds=10, verbose_eval=100)

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat0', 'cat1', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=95 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=95 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] Number of positive: 71549, number of negative: 198451
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.092057 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3390
[LightGBM] [Info] Number of data points in the train set: 270000, number of used features: 30
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=95 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.264996 -> initscore=-1.020160
[LightGBM] [Info] Start training from score -1.020160


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 10 rounds
[100]	valid_0's binary_logloss: 0.36097
[200]	valid_0's binary_logloss: 0.342214
[300]	valid_0's binary_logloss: 0.338063
[400]	valid_0's binary_logloss: 0.336583
Early stopping, best iteration is:
[463]	valid_0's binary_logloss: 0.33614


In [30]:
# ?gbm.train

In [31]:
# df_test = pd.read_csv(test_path)
# df_test.head()

In [32]:
df_test2 = df_test.drop(["id"], axis=1)

In [33]:
# for col in cat_col:
#     df_test2[col] = df_test2[col].apply(hash_cat_two)
#     print("done ", col)

In [34]:
# ?bst

In [35]:
df_test2.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10
0,0,5,0,0,5,33,0,8,23,0,...,0.735690,0.578366,0.723154,0.228037,0.356227,0.551249,0.655693,0.598331,0.359987,0.947489
1,0,7,2,0,4,2,7,31,51,0,...,0.313703,0.928885,0.516602,0.600169,0.795224,0.248987,0.654614,0.347944,0.565520,0.388580
2,0,13,2,0,5,2,0,8,28,0,...,0.448201,0.424876,0.344729,0.242073,0.270632,0.746740,0.335590,0.341238,0.252289,0.411592
3,1,11,2,0,5,33,0,30,23,0,...,0.666092,0.598943,0.561971,0.806347,0.735983,0.538724,0.381566,0.481660,0.348514,0.325723
4,0,5,0,1,5,33,0,8,46,0,...,0.772229,0.479572,0.767745,0.252454,0.354810,0.178920,0.763479,0.562491,0.466261,0.585781


In [36]:
pred = bst.predict(df_test2, num_iteration=bst.best_iteration)

In [37]:
df_test2

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10
0,0,5,0,0,5,33,0,8,23,0,...,0.735690,0.578366,0.723154,0.228037,0.356227,0.551249,0.655693,0.598331,0.359987,0.947489
1,0,7,2,0,4,2,7,31,51,0,...,0.313703,0.928885,0.516602,0.600169,0.795224,0.248987,0.654614,0.347944,0.565520,0.388580
2,0,13,2,0,5,2,0,8,28,0,...,0.448201,0.424876,0.344729,0.242073,0.270632,0.746740,0.335590,0.341238,0.252289,0.411592
3,1,11,2,0,5,33,0,30,23,0,...,0.666092,0.598943,0.561971,0.806347,0.735983,0.538724,0.381566,0.481660,0.348514,0.325723
4,0,5,0,1,5,33,0,8,46,0,...,0.772229,0.479572,0.767745,0.252454,0.354810,0.178920,0.763479,0.562491,0.466261,0.585781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0,13,0,3,5,33,0,6,23,0,...,0.361426,0.351946,0.327670,0.205547,0.679195,0.485967,0.319130,0.520681,0.519545,0.427119
199996,1,8,0,1,4,33,0,19,55,4,...,0.551106,0.628843,0.677765,0.624935,0.555306,0.242424,0.700829,0.531728,0.528427,0.922645
199997,0,11,3,0,7,33,0,14,47,0,...,0.812384,0.803348,0.324762,0.665624,0.488447,0.853213,0.578641,0.811941,0.537106,0.531758
199998,0,10,0,2,5,33,0,43,38,0,...,0.811282,0.820635,0.561449,0.797434,0.555089,0.746532,0.369986,0.438712,0.715524,0.381978


In [38]:
df_pred_final = df_test[["id"]]

In [39]:
df_pred_final["target"] = pred

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
df_pred_final.head()

,id,target
0,5,0.120812
1,6,0.362003
2,8,0.023869
3,9,0.171144
4,11,0.111637


In [41]:
df_pred_final.to_csv('submission.csv', index=False)

In [42]:
pred

array([0.12081153, 0.36200327, 0.02386933, ..., 0.59942974, 0.07801134,
       0.52702348])

# evaluation local

In [43]:
from sklearn.metrics import roc_auc_score

In [44]:
# y_test_label = df_test["target"]

In [45]:
# roc_auc_score(y_test_label, pred)

# local test result 
auc = 0.8969435887822527